In [1]:
import pandas as pd
import open3d as o3d
import numpy as np
import csv
import copy
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
# ZED_data = '../data/20221210_data/ZED/vga/depth_PNG_3029_376_06-12-2022-19-40-27.png'
ZED_data = '../data/ZED/vga/point_cloud_PLY_3029_376_06-12-2022-19-40-31.ply'
LiDAR_data = '../data/lidar/300/2022-12-10-16-53-36_Velodyne-VLP-16-Data.csv'

os.path.exists(ZED_data)
os.path.exists(LiDAR_data)

True

In [16]:
pcd_zed = o3d.io.read_point_cloud(ZED_data)
pts_zed = np.asarray(pcd_zed.points)

print(pts_zed)
o3d.visualization.draw_geometries([pcd_zed])

[Open3D WARNING] Read PLY failed: unable to read file: ../data/ZED/vga/point_cloud_PLY_3029_376_06-12-2022-19-40-31.ply
[[-2.492405  1.447196 -2.436705]
 [-2.475858  1.441786 -2.427595]
 [-2.469097  1.442058 -2.428054]
 ...
 [ 1.574756 -0.893204 -1.709091]
 [ 1.579812 -0.893185 -1.709056]
 [ 1.584903 -0.893186 -1.709058]]


In [19]:
with open(LiDAR_data, newline='') as f:
    rows = list(csv.reader(f, delimiter=',', quotechar='"'))
    pts_lidar = np.zeros(shape=(len(rows) - 1, 3))
    headers = rows.pop(0)
    for index, row in enumerate(rows[1:]):
        # print(index, row[0])
        pts_lidar[index] = row[0:3]

In [20]:
coords = o3d.geometry.TriangleMesh.create_coordinate_frame()
pcd_lidar = o3d.geometry.PointCloud()
pcd_lidar.points = o3d.utility.Vector3dVector(pts_lidar)

In [23]:
# print(pts_lidar)

pcd_lidar_r = copy.deepcopy(pcd_lidar).translate((0, 0, 0))
pcd_lidar_r.rotate(pcd_lidar_r.get_rotation_matrix_from_xyz((-np.pi / 2, 0, 0)), center=(0, 0, 0))
# print(np.asarray(pcd_lidar_r.points))

PointCloud with 17556 points.

In [24]:
pts_lidar_r = np.asarray(pcd_lidar_r.points)

shape_zed = np.shape(pts_zed)

print(np.fft.fft2(pts_lidar_r))

# shape_lidar = np.shape(pts_lidar_r)
# print(shape_zed[0])
# x = np.array(range(shape_zed[0]))
# y = np.array(range(3))
# pts_lidar_u = scipy.interpolate.interp2d(x, y, pts_lidar_r, kind='linear')

[[-57644.87567119    +0.j          26224.43245974-51816.23020104j
   26224.43245974+51816.23020104j]
 [ -9467.15384367-24406.01117422j   7770.59007411-25222.59840305j
    -382.95972829 -8762.88286381j]
 [  7244.97900002 +3711.06259746j    582.5196759 +13474.58726836j
   -2935.59830351 +2367.47198415j]
 ...
 [ -1335.10175784 +5212.55556258j  -3797.41561872 +1490.79400273j
     299.61888488 +1061.92606214j]
 [  7244.97900002 -3711.06259746j  -2935.59830351 -2367.47198415j
     582.5196759 -13474.58726836j]
 [ -9467.15384367+24406.01117422j   -382.95972829 +8762.88286381j
    7770.59007411+25222.59840305j]]


In [25]:
pcd_zed.paint_uniform_color([1, 0.706, 0])
pcd_lidar_r.paint_uniform_color([1, 0, 0])

PointCloud with 17556 points.

In [28]:
o3d.visualization.draw_geometries([
    # coords,
    pcd_zed,
    # pcd_lidar,
    pcd_lidar_r
])

KeyboardInterrupt: 

In [27]:
#Writing points with rows as the coordinates
p1_t = np.asarray(pcd_lidar.points)
p2_t = np.asarray(pcd_lidar_r.points) #Approx transformation is 90 degree rot over x-axis and +1 in Z axis

print(p1_t)
print(p2_t)

#Take transpose as columns should be the points
p1 = p1_t.transpose()
p2 = p2_t.transpose()

#Calculate centroids
p1_c = np.mean(p1, axis = 1).reshape((-1,1)) #If you don't put reshape then the outcome is 1D with no rows/colums and is interpeted as rowvector in next minus operation, while it should be a column vector
p2_c = np.mean(p2, axis = 1).reshape((-1,1))

#Subtract centroids
q1 = p1-p1_c
q2 = p2-p2_c

#Calculate covariance matrix
H=np.matmul(q1,q2.transpose())

#Calculate singular value decomposition (SVD)
U, X, V_t = np.linalg.svd(H) #the SVD of linalg gives you Vt

#Calculate rotation matrix
R = np.matmul(V_t.transpose(),U.transpose())

assert np.allclose(np.linalg.det(R), 1.0), "Rotation matrix of N-point registration not 1, see paper Arun et al."

#Calculate translation matrix
T = p2_c - np.matmul(R,p1_c)

#Check result
result = T + np.matmul(R,p1)
if np.allclose(result,p2):
    print("transformation is correct!")
else:
    print("transformation is wrong...")

[[ 1.30146686e-02  4.97122574e+00  8.67733657e-02]
 [ 1.17653925e-02  4.21315956e+00 -9.72688377e-01]
 [ 1.39323259e-02  4.98913383e+00  2.61470437e-01]
 ...
 [-3.76536814e-03  1.96126759e+00  2.40813822e-01]
 [-6.66596636e-04  1.90965867e+00  2.34476611e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[[ 1.30146686e-02  8.67733657e-02 -4.97122574e+00]
 [ 1.17653925e-02 -9.72688377e-01 -4.21315956e+00]
 [ 1.39323259e-02  2.61470437e-01 -4.98913383e+00]
 ...
 [-3.76536814e-03  2.40813822e-01 -1.96126759e+00]
 [-6.66596636e-04  2.34476611e-01 -1.90965867e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
transformation is correct!
